In [ ]:
# Section 1: Setup

import zipfile
from pathlib import Path
from lxml import etree
import pandas as pd
import re

# Define paths
DATA_DIR = Path(".")
ZIP_PATH = DATA_DIR / "drugbank_all_full_database.xml.zip"
XML_PATH = DATA_DIR / "full database.xml"

# Unzip if not already unzipped
if not XML_PATH.exists():
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)
    print("Unzipped XML file")

In [ ]:
# Section 2: Parse XML

with open(XML_PATH, "rb") as f:
    tree = etree.parse(f)

root = tree.getroot()
ns = {"db": "http://www.drugbank.ca"}

In [ ]:

# Section 3: Parse multiple fields, flatten later

raw_records = []
for drug in root.findall("db:drug", ns):
    try:
        record = {
            "name": drug.findtext("db:name", namespaces=ns),
            "indication": drug.findtext("db:indication", namespaces=ns),
            "mechanism": drug.findtext("db:mechanism-of-action", namespaces=ns),
            "pharmacodynamics": drug.findtext("db:pharmacodynamics", namespaces=ns),
            "toxicity": drug.findtext("db:toxicity", namespaces=ns),
            "categories": [cat.text for cat in drug.findall("db:categories/db:category/db:category", ns)],
            "atc_codes": [atc.get("code") for atc in drug.findall("db:atc-codes/db:atc-code", ns)],
            "kingdom": drug.findtext("db:classification/db:kingdom", namespaces=ns),
            "drug_class": drug.findtext("db:classification/db:class", namespaces=ns),
            "subclass": drug.findtext("db:classification/db:subclass", namespaces=ns),
        }
        raw_records.append(record)
    except Exception as e:
        print(f"Skipping drug due to error: {e}")

df = pd.DataFrame(raw_records)

In [5]:
# Section 4 – Field coverage (% non‑null)

print(df.columns.tolist())
print(len(raw_records))

def pct_nonnull(series):
    """Return % of non‑null rows in this Series."""
    return series.notnull().mean() * 100

print("Field Coverage:")
print(
    df[["indication", "mechanism", "pharmacodynamics", "toxicity"]]
    .apply(pct_nonnull)
)

# Check ATC and category availability
print("\nATC code availability:")
print(df["atc_codes"].apply(lambda x: len(x) > 0).value_counts(normalize=True))

print("\nCategory availability:")
print(df["categories"].apply(lambda x: len(x) > 0).value_counts(normalize=True))

# Sample records with multiple categories and ATC codes
print("\nSample categories and ATC codes:")
print(df[["name", "categories", "atc_codes"]].dropna().sample(5))

# Classification overview
print("\nSample classifications:")
print(df[["name", "kingdom", "drug_class", "subclass"]].dropna().sample(5))

['name', 'indication', 'mechanism', 'pharmacodynamics', 'toxicity', 'categories', 'atc_codes', 'kingdom', 'drug_class', 'subclass']
17430
Field Coverage:
indication          100.0
mechanism           100.0
pharmacodynamics    100.0
toxicity            100.0
dtype: float64

ATC code availability:
atc_codes
False    0.796328
True     0.203672
Name: proportion, dtype: float64

Category availability:
categories
True     0.576994
False    0.423006
Name: proportion, dtype: float64

Sample categories and ATC codes:
                                                    name  \
4559                                       Tetrabenazine   
14563                            Lactobacillus paracasei   
15840                                        Tinlarebant   
2968   {[7-(Difluoro-Phosphono-Methyl)-Naphthalen-2-Y...   
13535  Influenza A virus A/Kansas/14/2017 X-327 (H3N2...   

                                              categories  atc_codes  
4559   [Adrenergic Agents, Cytochrome P-450 CYP2D6 Su..